<a href="https://colab.research.google.com/github/Sanket758/AI-Chatbot-with-Tensorflow/blob/master/MyChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install nltk
import nltk
nltk.download('punkt')

You are using pip version 18.1, however version 21.3.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ReStart\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [4]:
from nltk.stem import WordNetLemmatizer
import nltk
lemmatizer = WordNetLemmatizer()
import json
import pickle
import warnings
warnings.filterwarnings('ignore')

In [5]:
 %tensorflow_version 1.x
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

UsageError: Line magic function `%tensorflow_version` not found.


# Preprocessing

In [6]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('./content/intents.json').read() # you can find the intents file here:https://github.com/Sanket758/AI-Chatbot-with-Tensorflow
intents = json.loads(data_file)

In [7]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [8]:
nltk.download('wordnet')

[nltk_data] Error loading wordnet: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [13]:
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

BadZipFile: File is not a zip file

In [19]:
import random
import numpy as np

# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")


BadZipFile: File is not a zip file

In [20]:
from tensorflow.python.framework import ops
ops.reset_default_graph()

In [26]:
from keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras.layers import Dropout

# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

NameError: name 'train_x' is not defined

In [26]:
from tensorflow.keras.optimizers import SGD

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [27]:
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")


Epoch 1/200
34/34 [==============================] - 2s 6ms/step - loss: 3.3442 - accuracy: 0.0592
Epoch 2/200
34/34 [==============================] - 0s 6ms/step - loss: 3.1530 - accuracy: 0.0947: 0s - loss: 3.1598 - accuracy: 0.09
Epoch 3/200
34/34 [==============================] - 0s 7ms/step - loss: 2.9031 - accuracy: 0.1657
Epoch 4/200
34/34 [==============================] - 0s 7ms/step - loss: 2.7242 - accuracy: 0.3018
Epoch 5/200
34/34 [==============================] - 0s 7ms/step - loss: 2.5671 - accuracy: 0.3077
Epoch 6/200
34/34 [==============================] - 0s 10ms/step - loss: 2.2624 - accuracy: 0.3669
Epoch 7/200
34/34 [==============================] - 0s 11ms/step - loss: 2.2215 - accuracy: 0.3550
Epoch 8/200
34/34 [==============================] - 0s 11ms/step - loss: 2.0842 - accuracy: 0.3787
Epoch 9/200
34/34 [==============================] - 0s 6ms/step - loss: 2.0399 - accuracy: 0.3964
Epoch 10/200
34/34 [==============================] - 0s 7ms/step - lo

In [28]:
from keras.models import load_model
model = load_model('chatbot_model.h5')

In [31]:
intents = json.loads(open('./content/intents.json').read())
words = pickle.load(open('./content/words.pkl','rb'))
classes = pickle.load(open('./content/classes.pkl','rb'))

In [32]:
# Utility Methods
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))
def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [33]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result
def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res

In [36]:
res = chatbot_response("c'est quoi la plateforme arrdellbee ? ")
print(res)

ARRDELBEE est une plateforme d'appropriation du dÃ©veloppement territorial crÃ©Ã©e par ARRDEL.


In [35]:
!pip freeze > requirements.txt